# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   278.962299 -0.798298  1234  278.164001  279.760596   556.328002   
6   983.369919 -0.090924  1234  983.278995  983.460844  1966.557990   
7   761.724782 -0.860541  1234  760.864241  762.585324  1521.728481   
8   673.596490 -0.636441  1234  672.960049  674.232931  1345.920099   
9   788.979562 -0.314811  1234  788.664751  789.294373  1577.329503   
10  908.498981 -0.949380  1234  907.549601  909.448360  1815.099202   
11  496.802289 -0.158490  1234  496.643799  496.960779   993.287598   
12  384.681569  0.560126  1234  385.241695  384.121443   770.483389   
13  475.323294 -0.242834  1234  475.080460  475.566128   950.160920   
14  268.154635  0.546586  1234  268.701221  267.608049   537.402442   

              f  
5    559.521193  
6   1966.921688  
7   1525.170647  
8   1348.465861  
9   1578.588745  
10  1818.896721  
11   993.921558  
12   768.242886  
13   951.132257  
14   535.216098

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -278.164001   279.760596
6   -983.278995   983.460844
7   -760.864241   762.585324
8   -672.960049   674.232931
9   -788.664751   789.294373
10  -907.549601   909.448360
11  -496.643799   496.960779
12  -385.241695   384.121443
13  -475.080460   475.566128
14  -268.701221   267.608049

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -278.164001   279.760596
6   -983.278995   983.460844
7   -760.864241   762.585324
8   -672.960049   674.232931
9   -788.664751   789.294373
10  -907.549601   909.448360
11  -496.643799   496.960779
12  -385.241695   384.121443
13  -475.080460   475.566128
14  -268.701221   267.608049

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-278.16400106,  279.7605964 ],
       [-983.27899489,  983.46084379],
       [-760.86424065,  762.58532364],
       [-672.96004928,  674.23293054],
       [-788.66475146,  789.29437252],
       [-907.54960108,  909.44836035],
       [-496.643799  ,  496.96077918],
       [-385.2416946 ,  384.12144302],
       [-475.0804602 ,  475.56612844],
       [-268.70122091,  267.60804919]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5     554.328002   -559.521193
6    1964.557990  -1966.921688
7    1519.728481  -1525.170647
8    1343.920099  -1348.465861
9    1575.329503  -1578.588745
10   1813.099202  -1818.896721
11    991.287598   -993.921558
12    768.483389   -768.242886
13    948.160920   -951.132257
14    535.402442   -535.216098

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf